<h1>OFD1: Simpson-Golabi-Behmel syndrome, type 2. Budny et al, 2006</h1>
<p>We will process <a href="https://pubmed.ncbi.nlm.nih.gov/16783569/" target="__blank">Budny, et al. (2006) A novel X-linked recessive mental retardation syndrome comprising macrocephaly and ciliary dysfunction is allelic to oral-facial-digital type I syndrome</a> in this notebook.</p>

In [1]:
import phenopackets as php
from google.protobuf.json_format import MessageToDict, MessageToJson
from google.protobuf.json_format import Parse, ParseDict
import pandas as pd
pd.set_option('display.max_colwidth', None) # show entire column contents, important!
from collections import defaultdict
from pyphetools.creation import *
from pyphetools.visualization import *
import importlib.metadata
__version__ = importlib.metadata.version("pyphetools")
print(f"Using pyphetools version {__version__}")

Using pyphetools version 0.5.8


<h2>Importing HPO data</h2>
<p>pyphetools uses the Human Phenotype Ontology (HPO) to encode phenotypic features. The recommended way of doing this is to ingest the hp.json file using HpoParser, which in turn creates an HpoConceptRecognizer object. </p>
<p>The HpoParser can accept a hpo_json_file argument if you want to use a specific file. If the argument is not passed, it will download the latext hp.json file from the HPO GitHub site and store it in a new subdirectory called hpo_data. It will not download the file if the file is already downloaded.</p>

In [2]:
parser = HpoParser()
hpo_cr = parser.get_hpo_concept_recognizer()
hpo_version = parser.get_version()
pmid = "PMID:16783569"
title = "A novel X-linked recessive mental retardation syndrome comprising macrocephaly and ciliary dysfunction is allelic to oral-facial-digital type I syndrome"
metadata = MetaData(created_by="ORCID:0000-0002-5648-2155", pmid=pmid, pubmed_title=title)
metadata.default_versions_with_hpo(version=hpo_version)

<h2>Importing the supplemental table</h2>
<p>Here, we use the pandas library to import this file (note that the Python package called openpyxl must be installed to read Excel files with pandas, although the library does not need to be imported in this notebook). pyphetools expects a pandas DataFrame as input, and users can choose any input format available for pandas include CSV, TSV, and Excel, or can use any other method to transform their input data into a Pandas DataFrame before using pyphetools.</p>

In [3]:
df = pd.read_excel('input/PMID_16783569.xlsx')

In [4]:
df.head()

,individual,sex,age,variant,Developmental delay,Abnormal respiratory system physiology,Macrocephaly,Recurrent respiratory infections,"Intellectual disability, severe",High palate,Low-set ears,Broad thumb,Brachydactyly,Obesity,Large for gestational age,Postaxial polydactyly,Inguinal hernia
0,1,male,11,c.2122_2125dupGAAG,+,+,+,+,+,+,+,+,+,+,-,NaN,NaN
1,2,male,0,c.2122_2125dupGAAG,+,+,+,+,+,NaN,NaN,NaN,NaN,NaN,+,+,NaN
2,3,male,0,c.2122_2125dupGAAG,+,+,+,+,NaN,NaN,NaN,NaN,+,NaN,NaN,NaN,+
3,4,male,0,c.2122_2125dupGAAG,+,+,+,+,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,male,0,c.2122_2125dupGAAG,+,+,+,+,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


<h2>Column mappers</h2>
<p>Please see the notebook "Create phenopackets from tabular data with individuals in rows" for explanations. In the following cell we create a dictionary for the ColumnMappers. Note that the code is identical except that we use the df.loc function to get the corresponding row data</p>

In [5]:
hpo_cr = parser.get_hpo_concept_recognizer()
generator = SimpleColumnMapperGenerator(df=df, observed='+', excluded='-', hpo_cr=hpo_cr)
column_mapper_d = generator.try_mapping_columns()

In [6]:
print(generator.get_mapped_columns())        

['Developmental delay', 'Abnormal respiratory system physiology', 'Macrocephaly', 'Recurrent respiratory infections', 'Intellectual disability, severe', 'High palate', 'Low-set ears', 'Broad thumb', 'Brachydactyly', 'Obesity', 'Large for gestational age', 'Postaxial polydactyly', 'Inguinal hernia']


In [7]:
print(generator.get_unmapped_columns())      

['individual', 'sex', 'age', 'variant']


<h2>Variant Data</h2>
<p>The OFD1 variant data (HGVS transcript) is listed in the Variant (hg19, NM_015133.4) column.</p>

In [8]:
genome = 'hg19'
default_genotype = 'hemizygous'
transcript='NM_003611.3'
varMapper = VariantColumnMapper(assembly=genome,column_name='variant', 
                                transcript=transcript, default_genotype=default_genotype)

<h1>Demographic data</h1>
<p>pyphetools can be used to capture information about age, sex, and individual identifiers. This information is stored in a map of "IndividualMapper" objects. Special treatment may be required for the indifiers, which may be used as the column names or row index.</p>

In [9]:
ageMapper = AgeColumnMapper.by_year('age')
ageMapper.preview_column(df['age'])

,original column contents,age
0,11,P11Y
1,0,P0Y


In [10]:
sexMapper = SexColumnMapper(male_symbol='male', female_symbol='female', column_name='sex')
sexMapper.preview_column(df['sex'])

,original column contents,sex
0,male,MALE
1,male,MALE
2,male,MALE
3,male,MALE
4,male,MALE
5,male,MALE
6,male,MALE
7,male,MALE
8,male,MALE


In [11]:
encoder = CohortEncoder(df=df, 
                        hpo_cr=hpo_cr, 
                        column_mapper_d=column_mapper_d, 
                        individual_column_name="individual", 
                        agemapper=ageMapper, 
                        sexmapper=sexMapper,
                        variant_mapper=varMapper, 
                        metadata=metadata,
                        pmid=pmid)
encoder.set_disease(disease_id='OMIM:300209', label='Simpson-Golabi-Behmel syndrome, type 2')

In [12]:
individuals = encoder.get_individuals()

https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg19/NM_003611.3%3Ac.2122_2125dupGAAG/NM_003611.3?content-type=application%2Fjson
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg19/NM_003611.3%3Ac.2122_2125dupGAAG/NM_003611.3?content-type=application%2Fjson
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg19/NM_003611.3%3Ac.2122_2125dupGAAG/NM_003611.3?content-type=application%2Fjson
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg19/NM_003611.3%3Ac.2122_2125dupGAAG/NM_003611.3?content-type=application%2Fjson
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg19/NM_003611.3%3Ac.2122_2125dupGAAG/NM_003611.3?content-type=application%2Fjson
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg19/NM_003611.3%3Ac.2122_2125dupGAAG/NM_003611.3?content-type=application%2Fjson
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg19/NM_003611.3%3Ac.2122_2125du

In [13]:
i1 = individuals[-1]
phenopacket1 = i1.to_ga4gh_phenopacket(metadata=metadata.to_ga4gh())
json_string = MessageToJson(phenopacket1)
print(json_string)

{
  "id": "PMID_16783569_9",
  "subject": {
    "id": "9",
    "timeAtLastEncounter": {
      "age": {
        "iso8601duration": "P0Y"
      }
    },
    "sex": "MALE"
  },
  "phenotypicFeatures": [
    {
      "type": {
        "id": "HP:0001263",
        "label": "Global developmental delay"
      }
    },
    {
      "type": {
        "id": "HP:0002795",
        "label": "Abnormal respiratory system physiology"
      }
    },
    {
      "type": {
        "id": "HP:0000256",
        "label": "Macrocephaly"
      }
    },
    {
      "type": {
        "id": "HP:0002205",
        "label": "Recurrent respiratory infections"
      }
    }
  ],
  "interpretations": [
    {
      "id": "9",
      "progressStatus": "SOLVED",
      "diagnosis": {
        "disease": {
          "id": "OMIM:300209",
          "label": "Simpson-Golabi-Behmel syndrome, type 2"
        },
        "genomicInterpretations": [
          {
            "subjectOrBiosampleId": "9",
            "interpretationStatus":

In [14]:
from IPython.display import HTML, display

phenopackets = [i.to_ga4gh_phenopacket(metadata=metadata.to_ga4gh()) for i in individuals]
table = PhenopacketTable(phenopacket_list=phenopackets)
display(HTML(table.to_html()))

In [15]:
output_directory = "phenopackets"
Individual.output_individuals_as_phenopackets(individual_list=individuals,
                                              pmid=pmid, 
                                              metadata=metadata.to_ga4gh(),
                                              outdir=output_directory)

We output 9 GA4GH phenopackets to the directory phenopackets
